# Experimental RL Optimization for DSPy

This notebook demonstrates how to use Reinforcement Learning (RL) techniques to optimize DSPy programs and improve their performance through trial-and-error learning.

Based on the DSPy tutorial: [Experimental RL Optimization for DSPy](https://dspy.ai/tutorials/rl_ai_program/)

## Setup

Import necessary libraries and configure the environment.

In [ ]:
import os
import sys
sys.path.append('../../')

import dspy
from utils import setup_default_lm, print_step, print_result, print_error
from utils.datasets import get_sample_qa_data, get_sample_classification_data
from dotenv import load_dotenv
import random
import numpy as np
from typing import List, Dict, Any, Tuple
import json
from collections import defaultdict, deque

# Load environment variables
load_dotenv('../../.env')

## Language Model Configuration

Set up DSPy with a language model for RL optimization experiments.

In [ ]:
print_step("Setting up Language Model", "Configuring DSPy for RL optimization")

try:
    lm = setup_default_lm(provider="openai", model="gpt-4o", max_tokens=1000)
    dspy.configure(lm=lm)
    print_result("Language model configured successfully!", "Status")
except Exception as e:
    print_error(f"Failed to configure language model: {e}")

## RL Environment for DSPy Programs

Create a reinforcement learning environment for optimizing DSPy programs.

In [ ]:
class DSPyRLEnvironment:
    """RL Environment for optimizing DSPy programs."""
    
    def __init__(self, base_program, train_examples, eval_examples, metric_func):
        self.base_program = base_program
        self.train_examples = train_examples
        self.eval_examples = eval_examples
        self.metric_func = metric_func
        
        # State representation
        self.current_state = self._get_initial_state()
        self.episode_rewards = []
        self.step_count = 0
        self.max_steps = 50
        
        # Action space - different optimization strategies
        self.action_space = {
            0: "adjust_temperature",
            1: "modify_prompt_structure", 
            2: "change_example_selection",
            3: "adjust_chain_of_thought",
            4: "modify_output_format",
            5: "ensemble_prediction"
        }
        
        # History tracking
        self.performance_history = []
        self.action_history = []
        
    def _get_initial_state(self):
        """Get initial state representation."""
        # Evaluate current program performance
        performance = self._evaluate_program(self.base_program)
        
        return {
            "performance_score": performance,
            "steps_taken": 0,
            "recent_actions": [],
            "program_complexity": self._estimate_complexity(),
            "convergence_trend": 0.0
        }
    
    def _evaluate_program(self, program):
        """Evaluate program performance on evaluation examples."""
        correct = 0
        total = 0
        
        for example in self.eval_examples[:10]:  # Use subset for speed
            try:
                prediction = program(**example.inputs())
                if self.metric_func(example, prediction):
                    correct += 1
                total += 1
            except:
                total += 1
        
        return correct / total if total > 0 else 0.0
    
    def _estimate_complexity(self):
        """Estimate program complexity (simplified)."""
        # In a real implementation, this would analyze the program structure
        return random.uniform(0.3, 0.7)
    
    def reset(self):
        """Reset environment for new episode."""
        self.current_state = self._get_initial_state()
        self.step_count = 0
        self.episode_rewards = []
        return self.current_state
    
    def step(self, action):
        """Take an action and return new state, reward, done."""
        
        self.step_count += 1
        
        # Apply action to modify program
        modified_program, action_success = self._apply_action(action)
        
        # Evaluate modified program
        new_performance = self._evaluate_program(modified_program)
        
        # Calculate reward
        reward = self._calculate_reward(new_performance, action, action_success)
        
        # Update state
        self.current_state = self._update_state(new_performance, action)
        
        # Check if episode is done
        done = (self.step_count >= self.max_steps or 
                new_performance > 0.95 or  # Excellent performance achieved
                len(self.episode_rewards) > 10 and all(r < 0 for r in self.episode_rewards[-5:]))  # Consistent poor performance
        
        # Record history
        self.performance_history.append(new_performance)
        self.action_history.append(action)
        self.episode_rewards.append(reward)
        
        return self.current_state, reward, done, {"performance": new_performance, "action_success": action_success}
    
    def _apply_action(self, action):
        """Apply an action to modify the DSPy program."""
        
        action_name = self.action_space[action]
        
        try:
            if action_name == "adjust_temperature":
                # Simulate adjusting model temperature
                modified_program = self._modify_temperature()
                return modified_program, True
                
            elif action_name == "modify_prompt_structure":
                # Simulate prompt engineering
                modified_program = self._modify_prompts()
                return modified_program, True
                
            elif action_name == "change_example_selection":
                # Simulate changing few-shot examples
                modified_program = self._change_examples()
                return modified_program, True
                
            elif action_name == "adjust_chain_of_thought":
                # Simulate CoT modifications
                modified_program = self._adjust_cot()
                return modified_program, True
                
            elif action_name == "modify_output_format":
                # Simulate output format changes
                modified_program = self._modify_output_format()
                return modified_program, True
                
            elif action_name == "ensemble_prediction":
                # Simulate ensemble methods
                modified_program = self._create_ensemble()
                return modified_program, True
                
            else:
                return self.base_program, False
                
        except Exception as e:
            print_error(f"Action failed: {e}")
            return self.base_program, False
    
    def _modify_temperature(self):
        """Simulate temperature adjustment."""
        # In practice, this would modify the actual LM configuration
        class ModifiedProgram:
            def __init__(self, base):
                self.base = base
                self.temperature_adjustment = random.uniform(-0.2, 0.2)
            
            def __call__(self, **kwargs):
                # Simulate effect of temperature change
                result = self.base(**kwargs)
                # Add some randomness to simulate temperature effect
                if hasattr(result, 'confidence'):
                    result.confidence = max(0.1, min(1.0, result.confidence + self.temperature_adjustment))
                return result
        
        return ModifiedProgram(self.base_program)
    
    def _modify_prompts(self):
        """Simulate prompt modifications."""
        class ModifiedProgram:
            def __init__(self, base):
                self.base = base
                self.prompt_modification = random.choice(["more_detailed", "concise", "structured"])
            
            def __call__(self, **kwargs):
                # Simulate prompt engineering effect
                result = self.base(**kwargs)
                # Randomly modify performance based on prompt changes
                performance_modifier = random.uniform(0.9, 1.1)
                return result
        
        return ModifiedProgram(self.base_program)
    
    def _change_examples(self):
        """Simulate changing few-shot examples."""
        class ModifiedProgram:
            def __init__(self, base):
                self.base = base
                self.example_selection = random.choice(["diverse", "similar", "hard_cases"])
            
            def __call__(self, **kwargs):
                return self.base(**kwargs)
        
        return ModifiedProgram(self.base_program)
    
    def _adjust_cot(self):
        """Simulate Chain of Thought adjustments."""
        class ModifiedProgram:
            def __init__(self, base):
                self.base = base
                self.cot_style = random.choice(["step_by_step", "reasoning_first", "conclusion_first"])
            
            def __call__(self, **kwargs):
                return self.base(**kwargs)
        
        return ModifiedProgram(self.base_program)
    
    def _modify_output_format(self):
        """Simulate output format modifications."""
        class ModifiedProgram:
            def __init__(self, base):
                self.base = base
                self.output_format = random.choice(["structured", "free_form", "json", "bullet_points"])
            
            def __call__(self, **kwargs):
                return self.base(**kwargs)
        
        return ModifiedProgram(self.base_program)
    
    def _create_ensemble(self):
        """Simulate ensemble creation."""
        class ModifiedProgram:
            def __init__(self, base):
                self.base = base
                self.ensemble_size = random.randint(2, 4)
            
            def __call__(self, **kwargs):
                # Simulate ensemble voting
                return self.base(**kwargs)
        
        return ModifiedProgram(self.base_program)
    
    def _calculate_reward(self, new_performance, action, action_success):
        """Calculate reward for the action taken."""
        
        if not action_success:
            return -0.5  # Penalty for failed actions
        
        # Performance improvement reward
        if self.performance_history:
            performance_delta = new_performance - self.performance_history[-1]
            reward = performance_delta * 10  # Scale the reward
        else:
            reward = new_performance * 5  # Initial performance reward
        
        # Bonus for high absolute performance
        if new_performance > 0.8:
            reward += 2.0
        elif new_performance > 0.6:
            reward += 1.0
        
        # Penalty for low performance
        if new_performance < 0.3:
            reward -= 1.0
        
        # Efficiency bonus (fewer steps to good performance)
        if new_performance > 0.7 and self.step_count < 10:
            reward += 1.0
        
        return reward
    
    def _update_state(self, new_performance, action):
        """Update state based on new performance and action."""
        
        # Calculate convergence trend
        if len(self.performance_history) >= 3:
            recent_perf = self.performance_history[-3:]
            trend = (recent_perf[-1] - recent_perf[0]) / 3
        else:
            trend = 0.0
        
        return {
            "performance_score": new_performance,
            "steps_taken": self.step_count,
            "recent_actions": self.action_history[-5:],  # Last 5 actions
            "program_complexity": self.current_state["program_complexity"],
            "convergence_trend": trend
        }

# Test the RL environment
print_step("Testing RL Environment")

# Create a simple test program
class SimpleQA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.Predict("question -> answer")
    
    def forward(self, question):
        return self.generate_answer(question=question)

# Simple metric
def qa_metric(example, prediction, trace=None):
    return example.answer.lower() in prediction.answer.lower()

# Get test data
qa_data = get_sample_qa_data()
train_data = qa_data[:3]
eval_data = qa_data[3:]

# Initialize environment
simple_qa = SimpleQA()
rl_env = DSPyRLEnvironment(simple_qa, train_data, eval_data, qa_metric)

# Test environment
initial_state = rl_env.reset()
print_result(f"Initial state: {initial_state}")

# Take a few test steps
for i in range(3):
    action = random.randint(0, 5)
    state, reward, done, info = rl_env.step(action)
    print(f"Step {i+1}: Action {action}, Reward {reward:.3f}, Performance {info['performance']:.3f}")

## Q-Learning Agent for DSPy Optimization

Implement a Q-Learning agent to optimize DSPy programs.

In [ ]:
class QLearningAgent:
    """Q-Learning agent for optimizing DSPy programs."""
    
    def __init__(self, state_size, action_size, learning_rate=0.1, discount_factor=0.95, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Q-table (simplified state representation)
        self.q_table = defaultdict(lambda: np.zeros(action_size))
        
        # Training statistics
        self.training_rewards = []
        self.training_performance = []
        self.episode_count = 0
    
    def _state_to_key(self, state):
        """Convert state dict to a key for Q-table."""
        # Discretize continuous values for Q-table
        perf_bucket = int(state["performance_score"] * 10)  # 0-10
        steps_bucket = min(state["steps_taken"] // 5, 10)   # 0-10
        trend_bucket = int((state["convergence_trend"] + 1) * 5)  # 0-10
        
        return (perf_bucket, steps_bucket, trend_bucket)
    
    def choose_action(self, state):
        """Choose action using epsilon-greedy policy."""
        state_key = self._state_to_key(state)
        
        if random.random() < self.epsilon:
            # Exploration
            return random.randint(0, self.action_size - 1)
        else:
            # Exploitation
            return np.argmax(self.q_table[state_key])
    
    def learn(self, state, action, reward, next_state, done):
        """Update Q-values using Q-learning update rule."""
        
        state_key = self._state_to_key(state)
        next_state_key = self._state_to_key(next_state)
        
        # Q-learning update
        current_q = self.q_table[state_key][action]
        
        if done:
            target_q = reward
        else:
            target_q = reward + self.discount_factor * np.max(self.q_table[next_state_key])
        
        # Update Q-value
        self.q_table[state_key][action] = current_q + self.learning_rate * (target_q - current_q)
        
        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def train(self, environment, num_episodes=100):
        """Train the agent in the given environment."""
        
        print_step("Q-Learning Training", f"Training for {num_episodes} episodes")
        
        for episode in range(num_episodes):
            state = environment.reset()
            total_reward = 0
            step_count = 0
            
            while True:
                # Choose action
                action = self.choose_action(state)
                
                # Take action
                next_state, reward, done, info = environment.step(action)
                
                # Learn from experience
                self.learn(state, action, reward, next_state, done)
                
                # Update tracking
                total_reward += reward
                step_count += 1
                state = next_state
                
                if done:
                    break
            
            # Record episode statistics
            self.training_rewards.append(total_reward)
            final_performance = info.get("performance", 0.0)
            self.training_performance.append(final_performance)
            self.episode_count += 1
            
            # Print progress
            if episode % 20 == 0:
                avg_reward = np.mean(self.training_rewards[-20:]) if len(self.training_rewards) >= 20 else total_reward
                avg_performance = np.mean(self.training_performance[-20:]) if len(self.training_performance) >= 20 else final_performance
                print(f"Episode {episode}: Avg Reward: {avg_reward:.3f}, Avg Performance: {avg_performance:.3f}, Epsilon: {self.epsilon:.3f}")
        
        print_result(f"Training completed! Final epsilon: {self.epsilon:.3f}")
    
    def get_training_stats(self):
        """Get training statistics."""
        return {
            "total_episodes": self.episode_count,
            "final_epsilon": self.epsilon,
            "average_reward": np.mean(self.training_rewards) if self.training_rewards else 0,
            "best_performance": max(self.training_performance) if self.training_performance else 0,
            "final_performance": self.training_performance[-1] if self.training_performance else 0,
            "q_table_size": len(self.q_table)
        }

# Initialize and train Q-learning agent
print_step("Initializing Q-Learning Agent")

agent = QLearningAgent(
    state_size=4,  # Simplified state representation
    action_size=6,  # Number of available actions
    learning_rate=0.1,
    discount_factor=0.95,
    epsilon=1.0,
    epsilon_decay=0.995,
    epsilon_min=0.01
)

# Train the agent
agent.train(rl_env, num_episodes=50)

# Get training statistics
stats = agent.get_training_stats()
print_step("Training Statistics")
for key, value in stats.items():
    print(f"  {key}: {value}")

## Advanced RL Optimization with Policy Gradient

Implement a more sophisticated Policy Gradient method for DSPy optimization.

In [ ]:
class PolicyGradientAgent:
    """Policy Gradient agent for DSPy optimization."""
    
    def __init__(self, state_size, action_size, learning_rate=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        
        # Policy parameters (simplified linear policy)
        self.policy_weights = np.random.randn(state_size, action_size) * 0.1
        
        # Episode memory
        self.episode_states = []
        self.episode_actions = []
        self.episode_rewards = []
        
        # Training statistics
        self.episode_returns = []
        self.policy_losses = []
    
    def _state_to_vector(self, state):
        """Convert state dict to feature vector."""
        return np.array([
            state["performance_score"],
            state["steps_taken"] / 50.0,  # Normalize
            state["convergence_trend"],
            len(state["recent_actions"]) / 5.0  # Normalize
        ])
    
    def _softmax(self, x):
        """Softmax activation function."""
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x)
    
    def choose_action(self, state):
        """Choose action based on policy probabilities."""
        state_vector = self._state_to_vector(state)
        
        # Compute action probabilities
        logits = np.dot(state_vector, self.policy_weights)
        action_probs = self._softmax(logits)
        
        # Sample action from probability distribution
        action = np.random.choice(self.action_size, p=action_probs)
        
        return action, action_probs[action]
    
    def store_transition(self, state, action, reward):
        """Store transition for later learning."""
        state_vector = self._state_to_vector(state)
        
        self.episode_states.append(state_vector)
        self.episode_actions.append(action)
        self.episode_rewards.append(reward)
    
    def learn_from_episode(self):
        """Learn from completed episode using policy gradient."""
        
        if not self.episode_rewards:
            return
        
        # Calculate discounted returns
        returns = []
        G = 0
        for reward in reversed(self.episode_rewards):
            G = reward + 0.99 * G  # Discount factor = 0.99
            returns.insert(0, G)
        
        # Normalize returns
        returns = np.array(returns)
        if len(returns) > 1:
            returns = (returns - np.mean(returns)) / (np.std(returns) + 1e-8)
        
        # Policy gradient update
        for i in range(len(self.episode_states)):
            state = self.episode_states[i]
            action = self.episode_actions[i]
            G = returns[i]
            
            # Compute gradient
            logits = np.dot(state, self.policy_weights)
            action_probs = self._softmax(logits)
            
            # Policy gradient
            grad = np.zeros_like(self.policy_weights)
            for a in range(self.action_size):
                if a == action:
                    grad[:, a] = state * (1 - action_probs[a]) * G
                else:
                    grad[:, a] = -state * action_probs[a] * G
            
            # Update policy weights
            self.policy_weights += self.learning_rate * grad
        
        # Record episode return
        episode_return = sum(self.episode_rewards)
        self.episode_returns.append(episode_return)
        
        # Clear episode memory
        self.episode_states = []
        self.episode_actions = []
        self.episode_rewards = []
    
    def train(self, environment, num_episodes=100):
        """Train the policy gradient agent."""
        
        print_step("Policy Gradient Training", f"Training for {num_episodes} episodes")
        
        for episode in range(num_episodes):
            state = environment.reset()
            episode_reward = 0
            
            while True:
                # Choose action
                action, action_prob = self.choose_action(state)
                
                # Take action
                next_state, reward, done, info = environment.step(action)
                
                # Store transition
                self.store_transition(state, action, reward)
                
                episode_reward += reward
                state = next_state
                
                if done:
                    break
            
            # Learn from episode
            self.learn_from_episode()
            
            # Print progress
            if episode % 20 == 0:
                avg_return = np.mean(self.episode_returns[-20:]) if len(self.episode_returns) >= 20 else episode_reward
                avg_performance = np.mean([info.get("performance", 0) for _ in range(min(20, len(self.episode_returns)))])
                print(f"Episode {episode}: Avg Return: {avg_return:.3f}, Recent Performance: {info.get('performance', 0):.3f}")
        
        print_result("Policy Gradient training completed!")
    
    def get_policy_summary(self):
        """Get summary of learned policy."""
        return {
            "policy_weights_shape": self.policy_weights.shape,
            "total_episodes": len(self.episode_returns),
            "average_return": np.mean(self.episode_returns) if self.episode_returns else 0,
            "best_return": max(self.episode_returns) if self.episode_returns else 0,
            "policy_weights_norm": np.linalg.norm(self.policy_weights)
        }

# Test Policy Gradient agent
print_step("Testing Policy Gradient Agent")

pg_agent = PolicyGradientAgent(
    state_size=4,
    action_size=6,
    learning_rate=0.01
)

# Create fresh environment for PG training
rl_env_pg = DSPyRLEnvironment(simple_qa, train_data, eval_data, qa_metric)

# Train policy gradient agent
pg_agent.train(rl_env_pg, num_episodes=50)

# Get policy summary
pg_stats = pg_agent.get_policy_summary()
print_step("Policy Gradient Statistics")
for key, value in pg_stats.items():
    print(f"  {key}: {value}")

## Multi-Agent RL for Complex DSPy Optimization

Implement multiple RL agents working together to optimize different aspects of DSPy programs.

In [ ]:
class MultiAgentRLOptimizer:
    """Multi-agent RL system for comprehensive DSPy optimization."""
    
    def __init__(self, base_program, train_examples, eval_examples, metric_func):
        self.base_program = base_program
        self.train_examples = train_examples
        self.eval_examples = eval_examples
        self.metric_func = metric_func
        
        # Specialized agents for different optimization aspects
        self.agents = {
            "prompt_optimizer": QLearningAgent(state_size=4, action_size=3, learning_rate=0.1),
            "example_selector": QLearningAgent(state_size=4, action_size=3, learning_rate=0.1),
            "structure_optimizer": PolicyGradientAgent(state_size=4, action_size=3, learning_rate=0.01)
        }
        
        # Specialized action spaces for each agent
        self.agent_actions = {
            "prompt_optimizer": {
                0: "detailed_prompts",
                1: "concise_prompts", 
                2: "structured_prompts"
            },
            "example_selector": {
                0: "diverse_examples",
                1: "similar_examples",
                2: "hard_examples"
            },
            "structure_optimizer": {
                0: "chain_of_thought",
                1: "direct_prediction",
                2: "ensemble_approach"
            }
        }
        
        # Coordination mechanism
        self.coordination_history = []
        self.collaborative_rewards = []
    
    def optimize_collaboratively(self, num_episodes=30):
        """Optimize using collaborative multi-agent approach."""
        
        print_step("Multi-Agent Collaborative Optimization", f"Running {num_episodes} episodes")
        
        for episode in range(num_episodes):
            print_step(f"Episode {episode + 1}")
            
            # Each agent contributes to the optimization
            agent_contributions = {}
            total_episode_reward = 0
            
            for agent_name, agent in self.agents.items():
                print(f"  {agent_name} optimization...")
                
                # Create specialized environment for this agent
                specialized_env = self._create_specialized_environment(agent_name)
                
                # Run optimization for this agent
                state = specialized_env.reset()
                agent_reward = 0
                
                for step in range(5):  # Limited steps per agent per episode
                    if isinstance(agent, QLearningAgent):
                        action = agent.choose_action(state)
                    else:  # PolicyGradientAgent
                        action, _ = agent.choose_action(state)
                    
                    next_state, reward, done, info = specialized_env.step(action)
                    
                    # Learn from experience
                    if isinstance(agent, QLearningAgent):
                        agent.learn(state, action, reward, next_state, done)
                    else:
                        agent.store_transition(state, action, reward)
                    
                    agent_reward += reward
                    state = next_state
                    
                    if done:
                        break
                
                # For Policy Gradient agents, learn from episode
                if isinstance(agent, PolicyGradientAgent):
                    agent.learn_from_episode()
                
                agent_contributions[agent_name] = {
                    "reward": agent_reward,
                    "performance": info.get("performance", 0),
                    "actions_taken": step + 1
                }
                
                total_episode_reward += agent_reward
            
            # Coordinate agents and share rewards
            coordination_bonus = self._calculate_coordination_bonus(agent_contributions)
            total_episode_reward += coordination_bonus
            
            self.collaborative_rewards.append(total_episode_reward)
            self.coordination_history.append(agent_contributions)
            
            # Print episode summary
            if episode % 10 == 0:
                avg_reward = np.mean(self.collaborative_rewards[-10:]) if len(self.collaborative_rewards) >= 10 else total_episode_reward
                print(f"  Episode {episode}: Avg Collaborative Reward: {avg_reward:.3f}")
                
                for agent_name, contrib in agent_contributions.items():
                    print(f"    {agent_name}: Reward {contrib['reward']:.3f}, Performance {contrib['performance']:.3f}")
        
        print_result("Multi-agent optimization completed!")
    
    def _create_specialized_environment(self, agent_name):
        """Create specialized environment for specific agent."""
        
        # Simplified specialized environments
        if agent_name == "prompt_optimizer":
            # Focus on prompt-related actions
            specialized_env = DSPyRLEnvironment(self.base_program, self.train_examples, self.eval_examples, self.metric_func)
            specialized_env.action_space = {0: "detailed_prompts", 1: "concise_prompts", 2: "structured_prompts"}
        
        elif agent_name == "example_selector":
            # Focus on example selection
            specialized_env = DSPyRLEnvironment(self.base_program, self.train_examples, self.eval_examples, self.metric_func)
            specialized_env.action_space = {0: "diverse_examples", 1: "similar_examples", 2: "hard_examples"}
        
        else:  # structure_optimizer
            # Focus on program structure
            specialized_env = DSPyRLEnvironment(self.base_program, self.train_examples, self.eval_examples, self.metric_func)
            specialized_env.action_space = {0: "chain_of_thought", 1: "direct_prediction", 2: "ensemble_approach"}
        
        return specialized_env
    
    def _calculate_coordination_bonus(self, agent_contributions):
        """Calculate bonus reward for good coordination between agents."""
        
        # Reward agents for complementary improvements
        performances = [contrib["performance"] for contrib in agent_contributions.values()]
        avg_performance = np.mean(performances)
        
        # Bonus for high average performance
        coordination_bonus = 0.0
        if avg_performance > 0.7:
            coordination_bonus += 2.0
        elif avg_performance > 0.5:
            coordination_bonus += 1.0
        
        # Bonus for balanced contributions (avoid one agent dominating)
        rewards = [contrib["reward"] for contrib in agent_contributions.values()]
        if len(rewards) > 1:
            reward_std = np.std(rewards)
            if reward_std < 2.0:  # Balanced contributions
                coordination_bonus += 1.0
        
        return coordination_bonus
    
    def get_optimization_summary(self):
        """Get summary of multi-agent optimization."""
        
        summary = {
            "total_episodes": len(self.collaborative_rewards),
            "average_collaborative_reward": np.mean(self.collaborative_rewards) if self.collaborative_rewards else 0,
            "best_episode_reward": max(self.collaborative_rewards) if self.collaborative_rewards else 0,
            "agent_statistics": {}
        }
        
        # Get statistics for each agent
        for agent_name, agent in self.agents.items():
            if isinstance(agent, QLearningAgent):
                agent_stats = agent.get_training_stats()
            else:  # PolicyGradientAgent
                agent_stats = agent.get_policy_summary()
            
            summary["agent_statistics"][agent_name] = agent_stats
        
        # Coordination effectiveness
        if self.coordination_history:
            avg_coordination = np.mean([
                np.mean([contrib["performance"] for contrib in episode.values()]) 
                for episode in self.coordination_history
            ])
            summary["average_coordination_performance"] = avg_coordination
        
        return summary

# Test multi-agent RL optimization
print_step("Testing Multi-Agent RL Optimization")

multi_agent_optimizer = MultiAgentRLOptimizer(
    base_program=simple_qa,
    train_examples=train_data,
    eval_examples=eval_data,
    metric_func=qa_metric
)

# Run collaborative optimization
multi_agent_optimizer.optimize_collaboratively(num_episodes=20)

# Get optimization summary
optimization_summary = multi_agent_optimizer.get_optimization_summary()

print_step("Multi-Agent Optimization Summary")
print(f"  Total Episodes: {optimization_summary['total_episodes']}")
print(f"  Average Collaborative Reward: {optimization_summary['average_collaborative_reward']:.3f}")
print(f"  Best Episode Reward: {optimization_summary['best_episode_reward']:.3f}")

if "average_coordination_performance" in optimization_summary:
    print(f"  Average Coordination Performance: {optimization_summary['average_coordination_performance']:.3f}")

print_step("Individual Agent Performance")
for agent_name, stats in optimization_summary["agent_statistics"].items():
    print(f"  {agent_name}:")
    for key, value in stats.items():
        print(f"    {key}: {value}")

## RL-Based Hyperparameter Optimization

Use RL to optimize DSPy hyperparameters automatically.

In [ ]:
class RLHyperparameterOptimizer:
    """RL-based hyperparameter optimization for DSPy programs."""
    
    def __init__(self, base_program_class, train_examples, eval_examples, metric_func):
        self.base_program_class = base_program_class
        self.train_examples = train_examples
        self.eval_examples = eval_examples
        self.metric_func = metric_func
        
        # Hyperparameter space
        self.hyperparameter_space = {
            "temperature": [0.0, 0.3, 0.7, 1.0],
            "max_tokens": [100, 300, 500, 1000],
            "num_examples": [1, 3, 5, 8],
            "optimization_method": ["basic", "bootstrap", "ensemble"]
        }
        
        # RL agent for hyperparameter selection
        self.hp_agent = QLearningAgent(
            state_size=4,  # Current performance, episode number, recent trend, complexity
            action_size=len(self._enumerate_hyperparameter_combinations()),
            learning_rate=0.15,
            epsilon=0.8,
            epsilon_decay=0.99
        )
        
        # Optimization history
        self.optimization_history = []
        self.best_hyperparameters = None
        self.best_performance = 0.0
    
    def _enumerate_hyperparameter_combinations(self):
        """Enumerate all possible hyperparameter combinations."""
        
        combinations = []
        
        for temp in self.hyperparameter_space["temperature"]:
            for tokens in self.hyperparameter_space["max_tokens"]:
                for examples in self.hyperparameter_space["num_examples"]:
                    for method in self.hyperparameter_space["optimization_method"]:
                        combinations.append({
                            "temperature": temp,
                            "max_tokens": tokens,
                            "num_examples": examples,
                            "optimization_method": method
                        })
        
        return combinations
    
    def _evaluate_hyperparameters(self, hyperparameters):
        """Evaluate a set of hyperparameters."""
        
        try:
            # Create program with hyperparameters
            program = self.base_program_class()
            
            # Simulate applying hyperparameters
            # In practice, you would configure the actual DSPy program
            performance_modifier = 1.0
            
            # Temperature effect
            if hyperparameters["temperature"] < 0.3:
                performance_modifier *= 0.95  # Conservative
            elif hyperparameters["temperature"] > 0.7:
                performance_modifier *= 0.9   # Too random
            
            # Token limit effect
            if hyperparameters["max_tokens"] < 200:
                performance_modifier *= 0.8   # Too restrictive
            elif hyperparameters["max_tokens"] > 800:
                performance_modifier *= 0.95  # Diminishing returns
            
            # Example count effect
            if hyperparameters["num_examples"] >= 5:
                performance_modifier *= 1.1   # Good few-shot learning
            elif hyperparameters["num_examples"] <= 1:
                performance_modifier *= 0.85  # Limited context
            
            # Optimization method effect
            if hyperparameters["optimization_method"] == "ensemble":
                performance_modifier *= 1.15
            elif hyperparameters["optimization_method"] == "bootstrap":
                performance_modifier *= 1.05
            
            # Evaluate on test data
            correct = 0
            total = 0
            
            for example in self.eval_examples[:5]:  # Use subset for speed
                try:
                    # Simulate prediction with hyperparameters
                    # In practice, this would use the configured program
                    base_prediction = program(**example.inputs())
                    
                    # Apply performance modifier
                    if random.random() < performance_modifier:
                        prediction_correct = self.metric_func(example, base_prediction)
                    else:
                        prediction_correct = False
                    
                    if prediction_correct:
                        correct += 1
                    total += 1
                    
                except:
                    total += 1
            
            performance = correct / total if total > 0 else 0.0
            
            return performance * performance_modifier
            
        except Exception as e:
            print_error(f"Hyperparameter evaluation failed: {e}")
            return 0.0
    
    def optimize_hyperparameters(self, num_episodes=50):
        """Optimize hyperparameters using RL."""
        
        print_step("RL Hyperparameter Optimization", f"Optimizing for {num_episodes} episodes")
        
        hyperparameter_combinations = self._enumerate_hyperparameter_combinations()
        
        for episode in range(num_episodes):
            # Current state (simplified)
            current_performance = self.best_performance
            episode_progress = episode / num_episodes
            recent_trend = self._calculate_recent_trend()
            complexity_estimate = 0.5  # Simplified
            
            state = {
                "performance_score": current_performance,
                "steps_taken": episode,
                "convergence_trend": recent_trend,
                "program_complexity": complexity_estimate
            }
            
            # Choose hyperparameter combination
            action = self.hp_agent.choose_action(state)
            
            if action < len(hyperparameter_combinations):
                hyperparameters = hyperparameter_combinations[action]
            else:
                # Fallback to random combination
                hyperparameters = random.choice(hyperparameter_combinations)
            
            # Evaluate hyperparameters
            performance = self._evaluate_hyperparameters(hyperparameters)
            
            # Calculate reward
            reward = self._calculate_hyperparameter_reward(performance, hyperparameters)
            
            # Update best performance
            if performance > self.best_performance:
                self.best_performance = performance
                self.best_hyperparameters = hyperparameters.copy()
            
            # Record history
            self.optimization_history.append({
                "episode": episode,
                "hyperparameters": hyperparameters,
                "performance": performance,
                "reward": reward,
                "is_best": performance == self.best_performance
            })
            
            # Learn from experience (simplified next state)
            next_state = {
                "performance_score": max(current_performance, performance),
                "steps_taken": episode + 1,
                "convergence_trend": self._calculate_recent_trend(),
                "program_complexity": complexity_estimate
            }
            
            done = episode == num_episodes - 1
            self.hp_agent.learn(state, action, reward, next_state, done)
            
            # Print progress
            if episode % 10 == 0:
                print(f"Episode {episode}: Performance {performance:.3f}, Best {self.best_performance:.3f}")
                print(f"  Hyperparameters: {hyperparameters}")
        
        print_result(f"Hyperparameter optimization completed!")
        print_result(f"Best performance: {self.best_performance:.3f}")
        print_result(f"Best hyperparameters: {self.best_hyperparameters}")
    
    def _calculate_recent_trend(self):
        """Calculate recent performance trend."""
        if len(self.optimization_history) < 3:
            return 0.0
        
        recent_performances = [h["performance"] for h in self.optimization_history[-3:]]
        return (recent_performances[-1] - recent_performances[0]) / 3
    
    def _calculate_hyperparameter_reward(self, performance, hyperparameters):
        """Calculate reward for hyperparameter choice."""
        
        # Base reward from performance
        reward = performance * 10
        
        # Bonus for improvement
        if performance > self.best_performance:
            improvement = performance - self.best_performance
            reward += improvement * 20  # Bonus for improvement
        
        # Efficiency bonus for good configurations
        if (hyperparameters["temperature"] == 0.3 and 
            hyperparameters["num_examples"] >= 3 and
            hyperparameters["optimization_method"] in ["bootstrap", "ensemble"]):
            reward += 2.0
        
        # Penalty for extreme configurations
        if hyperparameters["temperature"] > 0.8 or hyperparameters["max_tokens"] > 900:
            reward -= 1.0
        
        return reward
    
    def get_optimization_report(self):
        """Get comprehensive optimization report."""
        
        if not self.optimization_history:
            return {"error": "No optimization history available"}
        
        performances = [h["performance"] for h in self.optimization_history]
        
        report = {
            "total_episodes": len(self.optimization_history),
            "best_performance": self.best_performance,
            "best_hyperparameters": self.best_hyperparameters,
            "average_performance": np.mean(performances),
            "performance_std": np.std(performances),
            "improvement_episodes": len([h for h in self.optimization_history if h["is_best"]]),
            "final_epsilon": self.hp_agent.epsilon,
            "hyperparameter_analysis": self._analyze_hyperparameter_importance()
        }
        
        return report
    
    def _analyze_hyperparameter_importance(self):
        """Analyze which hyperparameters had the most impact."""
        
        analysis = {}
        
        for param_name in self.hyperparameter_space.keys():
            param_performance = defaultdict(list)
            
            for history_item in self.optimization_history:
                param_value = history_item["hyperparameters"][param_name]
                param_performance[param_value].append(history_item["performance"])
            
            # Calculate average performance for each parameter value
            param_averages = {
                value: np.mean(performances) 
                for value, performances in param_performance.items()
            }
            
            analysis[param_name] = param_averages
        
        return analysis

# Test RL hyperparameter optimization
print_step("Testing RL Hyperparameter Optimization")

hp_optimizer = RLHyperparameterOptimizer(
    base_program_class=SimpleQA,
    train_examples=train_data,
    eval_examples=eval_data,
    metric_func=qa_metric
)

# Run hyperparameter optimization
hp_optimizer.optimize_hyperparameters(num_episodes=30)

# Get optimization report
hp_report = hp_optimizer.get_optimization_report()

print_step("Hyperparameter Optimization Report")
print(f"  Best Performance: {hp_report['best_performance']:.3f}")
print(f"  Best Hyperparameters: {hp_report['best_hyperparameters']}")
print(f"  Average Performance: {hp_report['average_performance']:.3f}")
print(f"  Improvement Episodes: {hp_report['improvement_episodes']}")

print_step("Hyperparameter Importance Analysis")
for param_name, param_analysis in hp_report["hyperparameter_analysis"].items():
    print(f"  {param_name}:")
    sorted_values = sorted(param_analysis.items(), key=lambda x: x[1], reverse=True)
    for value, avg_performance in sorted_values[:2]:  # Top 2 values
        print(f"    {value}: {avg_performance:.3f}")

## Best Practices for RL Optimization in DSPy

### Key Principles:

1. **Environment Design**: Create meaningful state representations and reward functions
2. **Action Space**: Define actions that correspond to actual DSPy optimizations
3. **Exploration vs Exploitation**: Balance trying new approaches with using known good ones
4. **Multi-Agent Coordination**: Use specialized agents for different optimization aspects
5. **Hyperparameter Search**: Apply RL to automatically tune DSPy configurations

### RL Algorithm Selection:

- **Q-Learning**: Simple and effective for discrete action spaces
- **Policy Gradient**: Better for continuous or complex action spaces
- **Actor-Critic**: Combines benefits of value-based and policy-based methods
- **Multi-Agent RL**: Coordinate multiple optimization objectives

### Implementation Considerations:

- **State Representation**: Include performance metrics, optimization history, and program characteristics
- **Reward Engineering**: Design rewards that align with actual DSPy performance goals
- **Sample Efficiency**: Use sample-efficient algorithms to minimize LM API calls
- **Stability**: Implement safeguards against unstable optimization trajectories

### Production Deployment:

- **Online Learning**: Continue optimization during deployment
- **Safety Constraints**: Ensure RL doesn't degrade critical performance metrics
- **Monitoring**: Track RL agent behavior and intervention triggers
- **Rollback Mechanisms**: Ability to revert to previous good configurations

## Conclusion

This notebook demonstrated experimental RL optimization techniques for DSPy:

- **RL Environment**: Created environments for optimizing DSPy programs
- **Q-Learning**: Implemented value-based RL for discrete optimization choices
- **Policy Gradient**: Used policy-based methods for more complex optimization
- **Multi-Agent RL**: Coordinated multiple agents for comprehensive optimization
- **Hyperparameter Optimization**: Automated tuning of DSPy configurations

These experimental techniques show promise for:
- Automated optimization of complex DSPy programs
- Dynamic adaptation to changing requirements
- Multi-objective optimization (accuracy, speed, cost)
- Continuous improvement during deployment

While still experimental, RL-based optimization could become a powerful tool for automatically improving DSPy programs, especially for complex applications where manual optimization is challenging or time-consuming.